In [8]:
import pandas as pd
df=pd.read_csv('AQI_finaldataset.csv')
df = df.drop(['Location','CO','NO','NH3','O3'],axis=1)


In [9]:
df

,Location_ID,Date,AQI,NO2,SO2,PM2.5,PM10
0,0,2020-01-12,155,11.532500,14.119167,64.316667,71.244583
1,1,2020-01-12,175,17.677500,7.001667,101.994167,115.704583
2,2,2020-01-12,160,6.395833,3.274583,75.047500,81.250833
3,3,2020-01-12,160,9.455417,10.259167,74.220000,87.468750
4,4,2020-01-12,168,15.177917,6.458333,88.811250,98.957917
...,...,...,...,...,...,...,...
3690,0,2022-12-30,157,24.267500,19.873750,68.129583,76.675000
3691,1,2022-12-30,166,27.073750,14.355833,86.390000,99.132917
3692,2,2022-12-30,143,3.838333,2.935000,52.775417,58.132917
3693,3,2022-12-30,166,8.980833,12.270000,85.993333,98.587083


In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, Concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

In [ ]:

# Make predictions on test data
y_pred_scaled = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred_scaled)

# Calculate root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: %.3f' % rmse)
print(y_pred[0])

In [40]:
def train_dgc_lstm_model(df, location_id, model_name):
    # Filter data by location
    df = df[df["Location_ID"] == location_id]

    # Convert date column to datetime format
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")

    # Sort data by date
    df.sort_values("Date", inplace=True)

    # Extract target variables
    targets = ["AQI", "NO2", "SO2", "PM10", "PM2.5"]
    y = df[targets]

    # Normalize data
    scaler = MinMaxScaler()
    y_scaled = scaler.fit_transform(y)

    # Define sequence length for LSTM
    seq_length = 30

    # Create input and output sequences
    X, Y = [], []
    for i in range(len(y_scaled) - seq_length):
        X.append(y_scaled[i:i+seq_length])
        Y.append(y_scaled[i+seq_length])
    X = np.array(X)
    Y = np.array(Y)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Define input layer
    inputs = Input(shape=(seq_length, len(targets)))

    # Define spatial graph convolution layer
    for i in range(3):
        graph_conv = Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
        graph_conv = MaxPooling1D(pool_size=2)(graph_conv)
        graph_conv = BatchNormalization()(graph_conv)

        # Define temporal graph convolution layer
        temp_conv = Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
        temp_conv = MaxPooling1D(pool_size=2)(temp_conv)
        temp_conv = BatchNormalization()(temp_conv)

        # Concatenate graph convolutions
        merged = Concatenate(axis=-1)([graph_conv, temp_conv])

    # Define LSTM layer
    lstm = LSTM(128)(merged)

    # Define dense layers
    dense = Dense(64, activation='relu')(lstm)
    dense = Dropout(0.5)(dense)
    dense = Dense(len(targets))(dense)
    dense = Dense(64, activation='relu')(lstm)
    dense = Dropout(0.5)(dense)
    dense = Dense(len(targets))(dense)

    # Define output layer
    outputs = dense

    # Define model
    model = Model(inputs=inputs, outputs=outputs, name=model_name)

    # Compile model
    model.compile(loss='mse', optimizer='adam')

    # Train model
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
    model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test), callbacks=[es], verbose=2)

    return model

In [15]:
model_a1 = train_dgc_lstm_model(df, 1)

/var/folders/69/7vt__1z91mb3zslygkxndqpw0000gn/T/ipykernel_68188/1885808831.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
/Users/aman/tensorflow-test/env/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/100


2023-02-20 18:30:01.848900: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:30:02.177042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:30:02.269221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 - 2s - loss: 0.0996 - val_loss: 0.0883 - 2s/epoch - 113ms/step
Epoch 2/100


2023-02-20 18:30:02.954121: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:30:03.037964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 - 0s - loss: 0.0606 - val_loss: 0.0798 - 326ms/epoch - 18ms/step
Epoch 3/100
18/18 - 0s - loss: 0.0485 - val_loss: 0.0759 - 320ms/epoch - 18ms/step
Epoch 4/100
18/18 - 0s - loss: 0.0459 - val_loss: 0.0690 - 325ms/epoch - 18ms/step
Epoch 5/100
18/18 - 0s - loss: 0.0411 - val_loss: 0.0673 - 312ms/epoch - 17ms/step
Epoch 6/100
18/18 - 0s - loss: 0.0400 - val_loss: 0.0622 - 317ms/epoch - 18ms/step
Epoch 7/100
18/18 - 0s - loss: 0.0366 - val_loss: 0.0654 - 323ms/epoch - 18ms/step
Epoch 8/100
18/18 - 0s - loss: 0.0341 - val_loss: 0.0579 - 315ms/epoch - 17ms/step
Epoch 9/100
18/18 - 0s - loss: 0.0323 - val_loss: 0.0592 - 314ms/epoch - 17ms/step
Epoch 10/100
18/18 - 0s - loss: 0.0321 - val_loss: 0.0525 - 303ms/epoch - 17ms/step
Epoch 11/100
18/18 - 0s - loss: 0.0307 - val_loss: 0.0504 - 297ms/epoch - 17ms/step
Epoch 12/100
18/18 - 0s - loss: 0.0278 - val_loss: 0.0467 - 326ms/epoch - 18ms/step
Epoch 13/100
18/18 - 0s - loss: 0.0276 - val_loss: 0.0484 - 291ms/epoch - 16ms/step
Epoch 14/100

Epoch 100/100
18/18 - 0s - loss: 0.0093 - val_loss: 0.0279 - 309ms/epoch - 17ms/step


In [13]:
def predict_future(model, X_test, scaler, seq_length, targets, start_date, end_date, freq='D'):
    """
    Generates predictions for future dates using a trained LSTM model.

    Args:
    - model: a trained Keras LSTM model
    - X_test: a numpy array of shape (num_samples, seq_length, num_features) containing test data
    - scaler: a fitted sklearn.preprocessing.MinMaxScaler object used to scale the data
    - seq_length: the sequence length used for training the model
    - targets: a list of target column names
    - start_date: the start date for the future date range (string in 'YYYY-MM-DD' format)
    - end_date: the end date for the future date range (string in 'YYYY-MM-DD' format)
    - freq: the frequency of the future date range (default='D')

    Returns:
    - preds_df: a pandas DataFrame of shape (num_predictions, num_targets) containing the predicted values
    """
    # Define future date range
    future_dates = pd.date_range(start=start_date, end=end_date, freq=freq)

    # Create input sequences for future dates
    last_seq = np.array(X_test[-1])  # last sequence from the test data
    input_seq = last_seq.copy()
    preds = []
    for i in range(len(future_dates)):
        pred = model.predict(input_seq.reshape(1, seq_length, len(targets)))
        preds.append(pred)
        input_seq = np.append(input_seq[1:], pred, axis=0)

    # Inverse transform the predicted values
    preds = np.array(preds).reshape(-1, len(targets))
    preds_inv = scaler.inverse_transform(preds)

    # Convert predictions and dates to a pandas DataFrame
    preds_df = pd.DataFrame(preds_inv, index=future_dates, columns=targets)

    return preds_df



In [17]:
predict_future(model_a1, X_test, scaler, seq_length, targets, start_date='2023-02-01', end_date='2023-02-28', freq='D')

2023-02-20 18:31:07.794324: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:31:07.898232: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,AQI,NO2,SO2,PM10,PM2.5
2023-02-01,162.523788,11.167418,14.255235,93.550308,81.979271
2023-02-02,148.921844,9.537496,12.976024,84.865593,74.539513
2023-02-03,152.632736,9.872520,11.704534,82.854492,71.401192
2023-02-04,180.298477,11.798573,14.726301,106.969116,93.979752
2023-02-05,170.798859,11.148227,13.152823,101.216438,88.853577
2023-02-06,152.787964,11.065556,13.720207,90.383537,80.171448
2023-02-07,162.080917,10.013520,13.988982,91.313484,80.437233
2023-02-08,186.766342,10.476390,16.331343,108.023811,94.844765
2023-02-09,180.666489,10.571139,15.781747,102.292770,89.433189
2023-02-10,170.329193,10.640274,15.417902,97.506882,85.453720


In [19]:
model_a0 = train_dgc_lstm_model(df, 0)

/var/folders/69/7vt__1z91mb3zslygkxndqpw0000gn/T/ipykernel_68188/1885808831.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
/Users/aman/tensorflow-test/env/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/100


2023-02-20 18:37:04.509641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:37:05.010126: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:37:05.104594: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:37:05.884573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:37:05.974438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 - 2s - loss: 0.0871 - val_loss: 0.0569 - 2s/epoch - 135ms/step
Epoch 2/100
18/18 - 0s - loss: 0.0501 - val_loss: 0.0503 - 340ms/epoch - 19ms/step
Epoch 3/100
18/18 - 0s - loss: 0.0384 - val_loss: 0.0487 - 324ms/epoch - 18ms/step
Epoch 4/100
18/18 - 0s - loss: 0.0351 - val_loss: 0.0441 - 312ms/epoch - 17ms/step
Epoch 5/100
18/18 - 0s - loss: 0.0340 - val_loss: 0.0455 - 327ms/epoch - 18ms/step
Epoch 6/100
18/18 - 0s - loss: 0.0322 - val_loss: 0.0519 - 303ms/epoch - 17ms/step
Epoch 7/100
18/18 - 0s - loss: 0.0305 - val_loss: 0.0452 - 325ms/epoch - 18ms/step
Epoch 8/100
18/18 - 0s - loss: 0.0286 - val_loss: 0.0471 - 343ms/epoch - 19ms/step
Epoch 9/100
18/18 - 0s - loss: 0.0267 - val_loss: 0.0431 - 334ms/epoch - 19ms/step
Epoch 10/100
18/18 - 0s - loss: 0.0257 - val_loss: 0.0403 - 327ms/epoch - 18ms/step
Epoch 11/100
18/18 - 0s - loss: 0.0231 - val_loss: 0.0396 - 300ms/epoch - 17ms/step
Epoch 12/100
18/18 - 0s - loss: 0.0232 - val_loss: 0.0418 - 338ms/epoch - 19ms/step
Epoch 13/100
18

In [20]:
predict_future(model_a0, X_test, scaler, seq_length, targets, start_date='2023-02-01', end_date='2023-02-28', freq='D')

2023-02-20 18:37:50.462930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 18:37:50.596795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,AQI,NO2,SO2,PM10,PM2.5
2023-02-01,162.771133,8.578809,10.257969,74.333763,65.543640
2023-02-02,155.618668,9.642659,10.749367,65.164429,58.346062
2023-02-03,137.193817,10.482187,10.591032,66.760521,60.936653
2023-02-04,148.329849,9.368301,9.232942,65.932884,56.474411
2023-02-05,162.153366,12.445340,13.331022,84.649017,74.881264
2023-02-06,165.553268,11.889607,12.774716,86.200409,75.430260
2023-02-07,181.898666,13.350243,15.209957,104.040871,91.545937
2023-02-08,191.209564,10.657102,12.977812,97.556847,81.120544
2023-02-09,198.160217,11.220460,13.994368,98.291000,81.231293
2023-02-10,196.761093,9.032623,11.634832,88.886742,69.486313


In [34]:
location_map = {'Adilabad': 0, 
                'Karimnagar': 1, 
                'Khammam': 2, 
                'Nizamabad': 3, 
                'Warangal': 4}

In [41]:
models = {}

# Train models for each location
for location, location_id in location_map.items():
    model_name = f'model_{location}'
    models[location] = train_dgc_lstm_model(df, location_id, model_name)
    

/var/folders/69/7vt__1z91mb3zslygkxndqpw0000gn/T/ipykernel_68188/929710441.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
/Users/aman/tensorflow-test/env/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/100


2023-02-20 21:27:39.176607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 21:27:40.414085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 21:27:40.557543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 21:27:41.509253: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 21:27:41.621023: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 - 7s - loss: 0.0871 - val_loss: 0.0690 - 7s/epoch - 395ms/step
Epoch 2/100
18/18 - 0s - loss: 0.0471 - val_loss: 0.0682 - 368ms/epoch - 20ms/step
Epoch 3/100
18/18 - 0s - loss: 0.0384 - val_loss: 0.0657 - 317ms/epoch - 18ms/step
Epoch 4/100
18/18 - 0s - loss: 0.0338 - val_loss: 0.0610 - 322ms/epoch - 18ms/step
Epoch 5/100
18/18 - 0s - loss: 0.0302 - val_loss: 0.0560 - 324ms/epoch - 18ms/step
Epoch 6/100
18/18 - 0s - loss: 0.0285 - val_loss: 0.0586 - 309ms/epoch - 17ms/step
Epoch 7/100
18/18 - 0s - loss: 0.0293 - val_loss: 0.0567 - 300ms/epoch - 17ms/step
Epoch 8/100
18/18 - 0s - loss: 0.0264 - val_loss: 0.0541 - 318ms/epoch - 18ms/step
Epoch 9/100
18/18 - 0s - loss: 0.0247 - val_loss: 0.0541 - 310ms/epoch - 17ms/step
Epoch 10/100
18/18 - 0s - loss: 0.0235 - val_loss: 0.0546 - 315ms/epoch - 18ms/step
Epoch 11/100
18/18 - 0s - loss: 0.0224 - val_loss: 0.0508 - 359ms/epoch - 20ms/step
Epoch 12/100
18/18 - 0s - loss: 0.0223 - val_loss: 0.0518 - 306ms/epoch - 17ms/step
Epoch 13/100
18

/var/folders/69/7vt__1z91mb3zslygkxndqpw0000gn/T/ipykernel_68188/929710441.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
/Users/aman/tensorflow-test/env/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


...

18/18 - 2s - loss: 0.0981 - val_loss: 0.0967 - 2s/epoch - 125ms/step
Epoch 2/100
18/18 - 0s - loss: 0.0570 - val_loss: 0.0730 - 358ms/epoch - 20ms/step
Epoch 3/100
18/18 - 0s - loss: 0.0442 - val_loss: 0.0670 - 324ms/epoch - 18ms/step
Epoch 4/100
18/18 - 0s - loss: 0.0396 - val_loss: 0.0674 - 335ms/epoch - 19ms/step
Epoch 5/100
18/18 - 0s - loss: 0.0379 - val_loss: 0.0615 - 314ms/epoch - 17ms/step
Epoch 6/100
18/18 - 0s - loss: 0.0368 - val_loss: 0.0633 - 323ms/epoch - 18ms/step
Epoch 7/100
18/18 - 0s - loss: 0.0320 - val_loss: 0.0653 - 304ms/epoch - 17ms/step
Epoch 8/100
18/18 - 0s - loss: 0.0305 - val_loss: 0.0596 - 289ms/epoch - 16ms/step
Epoch 9/100
18/18 - 0s - loss: 0.0283 - val_loss: 0.0575 - 306ms/epoch - 17ms/step
Epoch 10/100
18/18 - 0s - loss: 0.0282 - val_loss: 0.0544 - 342ms/epoch - 19ms/step
Epoch 11/100
18/18 - 0s - loss: 0.0262 - val_loss: 0.0514 - 289ms/epoch - 16ms/step
Epoch 12/100
18/18 - 0s - loss: 0.0243 - val_loss: 0.0529 - 296ms/epoch - 16ms/step
Epoch 13/100
18

/var/folders/69/7vt__1z91mb3zslygkxndqpw0000gn/T/ipykernel_68188/929710441.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
/Users/aman/tensorflow-test/env/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Epoch 1/100


2023-02-20 21:29:15.456924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 21:29:15.861776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 21:29:15.965082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 21:29:16.853456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-20 21:29:16.962109: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 - 3s - loss: 0.0831 - val_loss: 0.0724 - 3s/epoch - 160ms/step
Epoch 2/100
18/18 - 0s - loss: 0.0485 - val_loss: 0.0617 - 401ms/epoch - 22ms/step
Epoch 3/100
18/18 - 0s - loss: 0.0386 - val_loss: 0.0572 - 322ms/epoch - 18ms/step
Epoch 4/100
18/18 - 0s - loss: 0.0350 - val_loss: 0.0534 - 320ms/epoch - 18ms/step
Epoch 5/100
18/18 - 0s - loss: 0.0323 - val_loss: 0.0591 - 310ms/epoch - 17ms/step
Epoch 6/100
18/18 - 0s - loss: 0.0342 - val_loss: 0.0583 - 306ms/epoch - 17ms/step
Epoch 7/100
18/18 - 0s - loss: 0.0303 - val_loss: 0.0538 - 310ms/epoch - 17ms/step
Epoch 8/100
18/18 - 0s - loss: 0.0296 - val_loss: 0.0476 - 330ms/epoch - 18ms/step
Epoch 9/100
18/18 - 0s - loss: 0.0275 - val_loss: 0.0470 - 309ms/epoch - 17ms/step
Epoch 10/100
18/18 - 0s - loss: 0.0274 - val_loss: 0.0432 - 310ms/epoch - 17ms/step
Epoch 11/100
18/18 - 0s - loss: 0.0254 - val_loss: 0.0397 - 292ms/epoch - 16ms/step
Epoch 12/100
18/18 - 0s - loss: 0.0261 - val_loss: 0.0431 - 312ms/epoch - 17ms/step
Epoch 13/100
18

In [51]:
models = {}

for location, location_id in location_map.items():
    model_name = f'model_{location}'
    models[location] = model_name
    
models

{'Adilabad': 'model_Adilabad',
 'Karimnagar': 'model_Karimnagar',
 'Khammam': 'model_Khammam',
 'Nizamabad': 'model_Nizamabad',
 'Warangal': 'model_Warangal'}

In [46]:
for loc,model in models.items():
    print(loc)
    forecast = predict_future(model, X_test, scaler, seq_length, targets, start_date='2023-02-01', end_date='2023-02-28', freq='D')
    print(forecast)

Adilabad
                   AQI        NO2        SO2        PM10      PM2.5
2023-02-01  145.585464  10.634912  11.447943   70.385742  61.724670
2023-02-02  142.335953  13.639072  13.685410   73.008522  71.411621
2023-02-03  153.112946  14.382212  14.045220   89.532478  81.721413
2023-02-04  140.909576  10.707196   9.811964   72.778259  65.019577
2023-02-05  141.576721  12.909809  13.336891   92.796135  86.440262
2023-02-06  151.155014  12.439368  13.014493   90.489906  82.493477
2023-02-07  162.420776  14.107489  14.952380   99.974678  93.190308
2023-02-08  169.944534  14.274529  16.027988  101.956329  94.628090
2023-02-09  168.333145  14.453926  16.159643   96.249832  89.529640
2023-02-10  174.442520  15.369645  17.698011   99.094559  90.749382
2023-02-11  172.343124  15.140398  16.940590   91.985481  84.594307
2023-02-12  173.700378  16.792500  19.455008   97.554932  88.698997
2023-02-13  169.686493  16.904932  18.662216   92.634834  85.101128
2023-02-14  166.764999  17.899858  20.6